In [2]:
from bs4 import BeautifulSoup
import io
import pandas as pd
from fuzzywuzzy import process
from twitter_dm import TwitterUser
from multiprocessing import Pool



In [3]:
pg = BeautifulSoup(io.open("../data/wapo_truth/Tracking all of President Trump’s false or misleading claims - Washington Post.htm"),"lxml")
data = []
for row in pg.find_all("div", {"class":"claim-row"}):
    claim = row.find("div", {"class":"claim"}).text[1:-1]
    repeated = row.find("span", {"class":"repeated-total"}).text if row.find("span", {"class":"repeated-total"}) else "0"
    repeated = repeated.replace("Repeated ","").replace(" times", "").replace(" time","")
    repeated = int(repeated)
    details = row.find("div", {"class": "details"})
    topic = details.find_all("span")[-2].text
    data.append({
            "text":claim,
            "repeated":repeated,
            "topic":topic
        })
      

data = pd.DataFrame(data)

In [4]:
u = TwitterUser()
u.populate_tweets_from_file("../data/trump_tweets_updated.json")

In [5]:
trump_tweets = {t.text : t.id_str for t in u.tweets}

del u
del pg

def get_tid_for_text(d):
    i, text = d
    if i % 10 == 0:
        print i
    to_return = []
    for x in process.extract(text, trump_tweets.keys(), limit=2):
        if x[1] >= 90:
            to_return.append(trump_tweets[x[0]])
    return (i, to_return)

p = Pool(8)

res = p.map(get_tid_for_text,enumerate(data.text))
p.close()
res

0
70
140
210
40
110
80
250
180
150
10
220
120
50
90
190
260
230
20
60
160
100
30
280
270
200
170
320
130
350
240
290
420
390
490
330
360
430
530
300
460
400
340
370
500
440
540
310
560
410
470
380
450
550
510
630
570
700
480
670
600
520
640
580
770
710
680
810
740
610
650
780
590
820
620
750
720
790
690
660
840
830
760
800
730
910
850
980
880
1050
950
1020
920
860
1090
990
890
1060
960
1030
930
870
1000
900
1070
970
1040
940
1010
1080


[(0, ['1057325717162213378']),
 (1, ['1057249169507803137']),
 (2, ['1057069393090355200']),
 (3, ['1057059603605831680']),
 (4, ['1057063101189836801']),
 (5, ['1056922209111994373']),
 (6, ['1056919064906469376']),
 (7, ['1056879122348195841', '1056880287974649856']),
 (8, ['1056592374607024130']),
 (9, []),
 (10, ['1055822810940129283']),
 (11, ['1055412328571850753']),
 (12, ['1055414972635926528']),
 (13, ['1055458320390217728']),
 (14, ['1055064484644089856']),
 (15, ['1055077740792160256']),
 (16, ['1054775397798178816']),
 (17, ['1054755401651179521']),
 (18, ['1054351078328885248']),
 (19, ['1054354059535269888']),
 (20, ['1054430376557535232']),
 (21, ['1054432329710608385']),
 (22, ['1054556416155910144']),
 (23, []),
 (24, ['1054091502585069568']),
 (25, ['1054088531629490178']),
 (26, ['1053682023066288128']),
 (27, ['1053338516501889025']),
 (28, ['1052883467430694912']),
 (29, ['1053008701026693122']),
 (30, ['1053013864244219904']),
 (31, ['1053076069564071936']),
 (32,

In [16]:
found = []
not_found = []
for r in res:
    for tid in r[1]:
        found.append({"wapo_topic":data.iloc[r[0],].topic,
                      "wapo_repeated":data.iloc[r[0],].repeated,
                      "tid":tid,
                      "wapo_isfalse":1})
    if not len(r[1]):
        not_found.append(r)


In [20]:
pd.DataFrame(found).to_csv("../data/wapo_tid_mapping.csv",index=False)